In [42]:
# -*- coding: utf-8 -*-

import numpy as np
import random
import math
import h5py
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import warnings

# disable FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

train_data_path='./Training_Data/'
output_path='./NN_out/'

nnstr='Ozone' #NN name string
hiddenlayers=(100,90,75) #set MLNN hidden layers (hidden layers will be reset for aaNN based on number of bands)

#radoption=2 #TOA rad data type, options: 1=Lt, 2=Lrc 
addnoise=0 #Flag for adding Gaussian noise NOTE: do not add noise for forward training
noiselevel=0 #Gaussian noise level, e.g. 1=1%

#sensor band infos
band=[302,312,320,340,380]
trainband=np.arange(5)
nrrs=5
aodidx=np.arange(5)

#number of sample in the scatter plot, if total number of data points is less than this, half of the data will be used for plotting
nplotsample=5000 
	
nband=len(band)
ntrainband=len(trainband)
naod=len(aodidx) 
      
#read in training data
print('Loading training dataset...')
for i in np.arange(5):
    if i==0:       
       par=np.loadtxt(train_data_path +'ML-input'+str(i+1)+'.txt')
    else:
       par=np.append(par,np.loadtxt(train_data_path +'ML-input'+str(i+1)+'.txt'),0)

print('Training dataset loaded.')

#remove negative values
idx=np.where(np.sum(par<0,axis=1)==0)[0]
par=par[idx,:]

#total number of training cases
ncase=len(par)
rad=np.zeros((ncase,1))

rad[:,0] = par[:,4]

#generate the gaussian noise for each band
if addnoise==1:
    fname_prefix=nnstr+'WiGN_p'+str(noiselevel)
    noise=np.random.normal(1,noiselevel/100,(ncase,1))    # only use 2 ratio
    rad=np.multiply(rad,noise) 
else:
    fname_prefix=nnstr+'WoGN' 
     
# NN training: Ozone & Cloud Optical Depth   
#trainingoption = itrain + 1

nnlayer = hiddenlayers
            
nlayer=len(nnlayer) # number of hidden layers
    
#create layer string for file name
layerstr=''
for i in np.arange(nlayer):
    if i<nlayer-1:
       layerstr=layerstr+str(nnlayer[i])+'X'
    else:
       layerstr=layerstr+str(nnlayer[i])

trainx=np.zeros((ncase,3))
trainy=np.zeros((ncase,2))


trainx[:,0]=np.cos(np.deg2rad(par[:,0])) # geometry: solar zenith
trainx[:,1]=np.log10(par[:,3]) # Irradiance 380
trainx[:,2]=np.log10(par[:,4]) # Ratio

trainy[:,0]=np.log10(par[:,1])
trainy[:,1]=np.log10(par[:,2])


#print(par)
print(trainx)
print(trainy)

Loading training dataset...
Training dataset loaded.
[[ 0.91354546 -0.93178107  1.45530937]
 [ 0.81915204 -0.85043228  1.71761878]
 [ 0.95630476 -1.26181453  1.35592248]
 ...
 [ 0.82903757 -1.24188034  2.12141577]
 [ 0.8660254  -1.04480592  1.51450403]
 [ 0.76604444 -1.24843691  1.66095514]]
[[-0.04686877  0.58204062]
 [ 0.0141844   0.39469678]
 [-0.11588464  0.9810163 ]
 ...
 [ 0.09808969  0.85367996]
 [-0.05178271  0.66900058]
 [-0.05873709  0.80207556]]
